### Load Libraries

In [2]:
import json
import csv
from collections import Counter 
from textstat.textstat import textstat
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Reading-wise Relative Difficulty

In [11]:
tokenizer = RegexpTokenizer(r'\w+')

with open('reading-1.json') as json_data:
    data = json.load(json_data)
    
label_row = [ 'reading_id', 'text', 'flesch_kincaid', 'unique_word_count' ]
reading_data = {}
for item in data:
    unique_word_list = list(map(lambda x: x.lower(), tokenizer.tokenize(item['text'])))
    reading = {
        'text': item['text'],
        'flesch_kincaid': textstat.flesch_reading_ease(item['text']),
        'unique_word_count': len(set(unique_word_list))
    }
    reading_data[item['_id']] = reading
reading_data

with open('data-usable-1.json') as json_data:
    data = json.load(json_data)
    
label_row = [ 'user_id', 'reading_id', 'performance_id', 'relative_difficulty', 'unknown_count', 'unique_word_count', 'total_count', 'flesch_kincaid' ]
data_rows = []
data_rows.append(label_row)
i = 0
total_counts = []
for user in data['users']:
    user_id = i
    i = i + 1
    j = 0
    total_count = 0
    for performance in user['performances']:
        reading_id = j
        performance_id = performance['performance']['_id']
        knowns = list(map(lambda x: x['content'].lower(),  performance['knonws']))
        unknown_count = len(list(set(knowns)))
        total_count += unknown_count
        j = j + 1
    total_counts.append(total_count)

i = 0 
for user in data['users']:
    user_id = i
    i = i + 1
    j = 0
    for performance in user['performances']:
        reading_id = j
        performance_id = performance['performance']['_id']
#         unknown_count = len(performance['knonws'])
        knowns = list(map(lambda x: x['content'].lower(),  performance['knonws']))
        unknown_count = len(list(set(knowns)))
        unique_word_count = reading_data[performance['performance']['readingId']]['unique_word_count']
        difficulty = reading_data[performance['performance']['readingId']]['flesch_kincaid']
        relative_difficulty = unknown_count/unique_word_count
        data_row = [user_id, reading_id, performance_id, relative_difficulty, unknown_count, unique_word_count, total_counts[user_id], difficulty]
        data_rows.append(data_row)
        j = j + 1

with open("rec-with-percentage-no-dup.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(data_rows)
    print('done')


done


In [ ]:
from pymongo import MongoClient
from bson.json_util import dumps
with open('data-usable-1.json') as json_data:
    data = json.load(json_data)
    
client = MongoClient('mongodb://localhost:3001')
db = client.meteor

_map = {
    "大学学位": 'bachelor',
    "硕士学位": 'master',
    "大学未完成": 'in_college',
    "博士学位": 'doctor',
    "高中未完成": 'in_high_school',
    "高中毕业": 'high_school_grad',
    "高中学位": 'high_school_grad',
    "专科": 'skill_school'
}

label_row = [ 'user_id', 'gender', 'age', 'grade' ]
data_rows = []
data_rows.append(label_row)
for user in data['users']:
    user_id = user['user']['_id']
    user_local = db.users.find_one({'_id': user_id})
    if user_local['profile']['gender'] == '男性':
        gender = 'M'
    else:
        gender = 'F'
    age = user_local['profile']['age'][:-2]
    grade = _map[user_local['profile']['grade']]
    data_row = [user_id, gender, age, grade]
    data_rows.append(data_row)
    
with open("user_basic_info.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(data_rows)
    print('done')

In [ ]:
from pymongo import MongoClient
from bson.json_util import dumps
with open('data-usable-1.json') as json_data:
    data = json.load(json_data)

label_row = [ 'user_id', 'reading_id', 'time_length' ]
data_rows = []
data_rows.append(label_row)
for user_data in data['users']:
    user_id = user_data['user']['_id']
    for performance_data in user_data['performances']:
        performance = performance_data['performance']
        if 'endTime' in performance and 'startTime' in performance:
            diff = performance['endTime'] - performance['startTime']
        else:
            diff = 829332.0
        data_row = [user_id, performance['readingId'], diff]
        print(data_row)
        data_rows.append(data_row)
    
with open("user_reading_time.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(data_rows)
    print('done')

In [42]:
tokenizer = RegexpTokenizer(r'\w+')

with open('reading-1.json') as json_data:
    data = json.load(json_data)
    
label_row = [ 'reading_id', 'text', 'flesch_kincaid', 'unique_word_count' ]
reading_data = {}
for item in data:
    unique_word_list = list(map(lambda x: x.lower(), tokenizer.tokenize(item['text'])))
    reading = {
        'text': item['text'],
        'flesch_kincaid': textstat.flesch_reading_ease(item['text']),
        'unique_word_count': len(set(unique_word_list))
    }
    reading_data[item['_id']] = reading
reading_data

with open('rank_user.json') as json_data:
    data = json.load(json_data)
    
label_row = [ 'user_id', 'reading_id', 'performance_id', 'relative_difficulty', 'unknown_count', 'unique_word_count' ]
data_rows = []
data_rows.append(label_row)
i = 0
total_counts = []
for user in data['users']:
    user_id = i
    i = i + 1
    j = 0
    total_count = 0
    for performance in user['performances']:
        reading_id = j
        performance_id = performance['performance']['_id']
#         knowns = list(map(lambda x: x['content'].lower(),  performance['knonws']))
        knowns = performance['knonws']
        unknown_count = len(list(set(knowns)))
        total_count += unknown_count
        j = j + 1
    total_counts.append(total_count)

i = 0 
for user in data['users']:
    user_id = i
    i = i + 1
    j = 0
    for performance in user['performances']:
        reading_id = j
        performance_id = performance['performance']['_id']
#         unknown_count = len(performance['knonws'])
#         knowns = list(map(lambda x: x['content'].lower(),  performance['knonws']))
        knowns = performance['knonws']
        unknown_count = len(list(set(knowns)))
        unique_word_count = reading_data[performance['performance']['readingId']]['unique_word_count']
        difficulty = reading_data[performance['performance']['readingId']]['flesch_kincaid']
        relative_difficulty = unknown_count/unique_word_count
        data_row = [user_id, reading_id, performance_id, relative_difficulty, unknown_count, unique_word_count, total_counts[user_id], difficulty]
        data_rows.append(data_row)
        j = j + 1

with open("test_rank_user.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(data_rows)
    print('done')


done
